In [1]:
from model import Deeplabv3
from keras.callbacks import ModelCheckpoint
import numpy as np
import os,csv
from keras.preprocessing import image
from keras import optimizers

Using TensorFlow backend.


In [2]:
colormap =[[0, 0, 0],
 [15, 15, 15],
 [16, 16, 16],
 [17, 17, 17],
 [18, 18, 18],
 [21, 21, 21],
 [23, 23, 23],
 [24, 24, 24],
 [30, 30, 30],
 [41, 41, 41],
 [42, 42, 42],
 [43, 43, 43],
 [45, 45, 45],
 [46, 46, 46],
 [47, 47, 47],
 [49, 49, 49],
 [50, 50, 50],
 [54, 54, 54],
 [56, 56, 56],
 [62, 62, 62],
 [65, 65, 65],
 [90, 90, 90],
 [255, 255, 255]]

cm2lbl = np.zeros(256 ** 3)  # 每个像素点有 0 ~ 255 的选择，RGB 三个通道
for i, cm in enumerate(colormap):
    cm2lbl[(cm[0] * 256 + cm[1]) * 256 + cm[2]] = i  # 建立索引

    
def image2label(im):
    # 输入三维的mask图片，返回二维的矩阵，对每一个像素进行标记(320,320,3)->(320,320)
    #如[0,0,0]标记为0,[128,0,0]标记为1
    data = np.array(im, dtype='int32')
    idx = (data[:, :, 0] * 256 + data[:, :, 1]) * 256 + data[:, :, 2]
    return np.array(cm2lbl[idx], dtype='int64')

In [3]:
def generateData(batch_size,train=True,root_dir='/input0/land_23_jpg'):
    '''
    输入的是整个数据的images和masks，以及每次训练的batch_size数目，返回的是经过数据增强的batch_size批次大小的数组.
    其中mask的维度为(None,224,224),每次读取batch_size大小的images和masks时，做了(None,224,224)->(None,224,224,23)
    的转化，然后再做了数据增强。
    '''

    fr=open('%s/meta.csv'%root_dir)
    all_list=[]
    reader=csv.reader(fr)
    for each in reader:
        all_list.append(each)
    all_list=all_list[1:]
    train_num=int(len(all_list)*0.9)
    if train:
#         train_list = all_list[:train_num]
        train_list=all_list
    else:
        #         train_num=int(len(all_list)*0.1)
        train_list = all_list[train_num:]
    while True:
        train_data = []
        train_label = []
        batch = 0
        for each in train_list:
            batch += 1
            img = image.load_img('%s/%s' % (root_dir, each[1]))
            img = image.img_to_array(img)
            img=img/255.
            label = image.load_img('%s/%s' % (root_dir, each[0]))
            label = image.img_to_array(label)
            train_data.append(img)
            train_label.append(label)
            if batch % batch_size == 0:
                train_data = np.array(train_data)
                train_label = np.array(train_label)
                mask_label = np.zeros(train_label[:, :, :, 0].shape)

                for j in range(train_label.shape[0]):
                    mask_label[j] = image2label(train_label[j])
                mask_one_hot = np.zeros(mask_label.shape + (23,))
                for i in range(23):
                    mask_one_hot[mask_label == i, i] = 1
                train_label = mask_one_hot.astype('int32')
                yield (train_data, train_label)

                train_data = []
                train_label = []
                batch = 0

In [4]:
model = Deeplabv3(weights=None, input_shape=(224, 224, 3), classes=23, backbone='xception')
optimizer = optimizers.RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
if not os.path.exists('./models'):
    os.mkdir('./models')
checkpoint = ModelCheckpoint('./models/deeplab_2.hdf5', monitor='acc', save_best_only=True, verbose=1)

model.load_weights('./models/deeplab.hdf5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 112, 112, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [5]:
history = model.fit_generator(generator=generateData(16, True), steps_per_epoch=440, epochs=20,
                                  validation_data=generateData(16, False), validation_steps=45, callbacks=[checkpoint])

Epoch 1/20
440/440 [==============================] - 311s 706ms/step - loss: 0.1044 - acc: 0.9605 - val_loss: 0.4182 - val_acc: 0.8867

Epoch 00001: acc improved from -inf to 0.96045, saving model to ./models/deeplab_2.hdf5
Epoch 2/20
440/440 [==============================] - 245s 556ms/step - loss: 0.1040 - acc: 0.9605 - val_loss: 0.3234 - val_acc: 0.9106

Epoch 00002: acc improved from 0.96045 to 0.96054, saving model to ./models/deeplab_2.hdf5
Epoch 3/20
440/440 [==============================] - 232s 528ms/step - loss: 0.1081 - acc: 0.9599 - val_loss: 0.2039 - val_acc: 0.9326

Epoch 00003: acc did not improve from 0.96054
Epoch 4/20
440/440 [==============================] - 231s 524ms/step - loss: 0.0950 - acc: 0.9639 - val_loss: 0.3832 - val_acc: 0.8932

Epoch 00004: acc improved from 0.96054 to 0.96391, saving model to ./models/deeplab_2.hdf5
Epoch 5/20
440/440 [==============================] - 233s 529ms/step - loss: 0.0982 - acc: 0.9626 - val_loss: 0.3071 - val_acc: 0.9098
